In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils import get_completion

model_id = "kreimben/CodeMind-gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aksid\.cache\huggingface\hub\models--kreimben--CodeMind-gemma-2b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [2]:
model = model.to('cuda')

In [3]:
%%time

text = f'''leetcode wordbreak'''

res = get_completion(
    text,
    system_prompt='You are a kind coding test teacher. Solve the given question and give the insight and approach.',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


CPU times: total: 2.39 s
Wall time: 13.1 s


In [6]:
answer = res.split('model')[-1]

In [7]:
from IPython.display import display, Markdown

# answer = answer.replace('\\\\', '\\')

display(Markdown(answer))

**Python**\n```\ndef wordBreak(self, s: str, words: List[str]) -> bool:\n        \n        dp = [False] * len(s)\n        \n        for i in range(len(s)):\n            for j in range(i, len(s)):\n                if s[i:j+1] in words:\n                    dp[i] = True\n                    break\n        \n        return dp[-1]\n```\n\n**C++**\n```\nclass Solution {\npublic:\n    bool wordBreak(string s, vector<string>& words) {\n        vector<bool> dp(s.size(), false);\n        dp[0] = true;\n        for (int i = 1; i < s.size(); i++) {\n            for (int j = i - 1; j >= 0; j--) {\n                if (s.substr(j, i - j) == words[j]) {\n                    dp[i] = true;\n                    break;\n                }\n            }\n        }\n        return dp[s.size() - 1];\n    }\n};\n```\n\n**Java**\n```\nclass Solution {\n    public boolean wordBreak(String s, List<String> words) {\n        boolean[] dp = new boolean[s.length + 1];\n        dp[0] = true;\n        for (int i = 1; i < s.length(); i++) {\n            for (int j = i - 1; j >= 0; j--) {\n                if (s.substring(j, i + 1).equals(words[j])) {\n                    dp[i] = true;\n                    break;\n                }\n            }\n        }\n        return dp[s.length() - 1];\n    }\n}\n```\n\n**C#**\n```\npublic class Solution {\n    public bool WordBreak(string s, List<string> words)